In [4]:
import os
import subprocess
import time
import random
import csv
import paramiko
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool
from datetime import datetime

# (1) ############################################## Different Cross-node delays generation #############################################################

def generate_delay_matrix(num_nodes, base_latency, max_additional_latency):
    delay_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                additional_latency = random.uniform(0, max_additional_latency)
                distance_factor = abs(i - j) / num_nodes
                simulated_latency = base_latency + additional_latency * distance_factor
                congestion_factor = random.uniform(0.5, 1.5)
                delay_matrix[i][j] = int(simulated_latency * congestion_factor)
    return delay_matrix

In [5]:
#(1) zero delay with parameters: 
delay_matrix1_zero = generate_delay_matrix(9, 0, 0) 
#(2) light delay with parameters: 
delay_matrix2_light = generate_delay_matrix(9, 5, 20) 
#(3) middle delay with parameters: 
delay_matrix3_mid = generate_delay_matrix(9, 5, 30)
#(4) heavy delay with parameters: 
delay_matrix4_mid = generate_delay_matrix(9, 5, 60)

In [6]:
with open('delay_matrix1_zero.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(delay_matrix1_zero)
with open('delay_matrix2_light.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(delay_matrix2_light)
with open('delay_matrix3_mid.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(delay_matrix3_mid)
with open('delay_matrix4_heavy.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(delay_matrix4_mid)

In [7]:
import pandas as pd


# Read the delay matrix from the CSV file
delay_matrix = pd.read_csv('delay_matrix2_light.csv', header=None)
delay_matrix

,0,1,2,3,4,5,6,7,8
0,0,7,7,13,5,10,8,15,9
1,4,0,3,5,7,4,13,6,7
2,5,5,0,7,6,6,5,4,9
3,3,12,7,0,6,5,5,16,11
4,14,13,6,8,0,6,11,13,11
5,11,8,8,6,9,0,3,7,9
6,6,5,4,12,9,8,0,9,6
7,7,23,10,10,8,11,4,0,8
8,11,6,19,6,3,8,6,2,0
